<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# <center> Agno Agents: Orchestrator-Worker Pattern </center>

In this tutorial, we'll explore orchestrator-worker agent workflows with Agno Teams.

Agent orchestration in Agno enables multiple specialized agents to collaborate dynamically, with only the most relevant one (or ones) activated based on the current task context. Instead of following a fixed sequence, agents are coordinated by a central Team that decides when and how each agent contributes. At the end of the interaction, results can be synthesized into a cohesive response.

Orchestrator workflows in Agno simplify this routing pattern through a Team configured with instructions, success criteria, and an LLM model. The team routes queries to the right agents, ensures consistency across responses, and allows you to monitor and debug the workflow. With Phoenix tracing, you get full visibility into the orchestration flow to see which agents engaged, when they were activated, and why.

In this example, we'll build a smart trip planning assistant where subtasks like destination research, hotel booking, and activity suggestions are dynamically delegated to the right specialized agent.

By the end of this tutorial, you’ll learn how to:
- Set up multiple specialized Agno agents in a Team
- Use the Team orchestrator to enable dynamic agent routing
- Trace and visualize agent interactions using Phoenix

⚠️ You'll need an OpenAI Key and a [free Phoenix Cloud account](https://app.arize.com/auth/phoenix/signup) for this tutorial

In [ ]:
%pip install -qqqqq arize-phoenix openinference-instrumentation-agno agno openinference-instrumentation-openai openai ddgs

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("🔑 Enter your Phoenix API key: ")

if "PHOENIX_COLLECTOR_ENDPOINT" not in os.environ:
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass("🔑 Enter your Phoenix Collector Endpoint: ")

# Configure Tracing

In [ ]:
from phoenix.otel import register

tracer_provider = register(project_name="agno-orchestrator-worker-agent", auto_instrument=True)

# Set up Agents and Tools

In [ ]:
from agno.agent import Agent
from agno.memory.v2.memory import Memory
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools

memory = Memory()

memory.clear()

# Flight agent with live search + structured JSON responses
flight_agent = Agent(
    name="FlightPlanner",
    model=OpenAIChat(id="gpt-4o", temperature=0.2),
    description="Suggests flight options, itineraries, and pricing.",
    instructions=[
        "Always provide flight options with airlines, duration, and price.",
        "Return responses as JSON with keys: airline, departure, arrival, price.",
    ],
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    memory=memory,
)

# Hotel agent with memory for user preferences
hotel_agent = Agent(
    name="HotelFinder",
    model=OpenAIChat(id="gpt-4o"),
    description="Recommends accommodations matching location, preferences, and budget.",
    instructions=["Always include at least 3 hotel options with price and location."],
    markdown=True,
    memory=memory,
)

# Activity agent with creative flair
activity_agent = Agent(
    name="ActivitySuggester",
    model=OpenAIChat(id="gpt-4o", temperature=0.8),
    description="Proposes activities and attractions tailored to the traveler’s interests.",
    instructions=[
        "Group activities by category: cultural, food, outdoors.",
        "Include both popular and hidden-gem recommendations.",
    ],
    markdown=True,
    memory=memory,
)

In [ ]:
from agno.team import Team

travel_team = Team(
    name="TravelTeam",
    mode="coordinate",
    members=[flight_agent, hotel_agent, activity_agent],
    model=OpenAIChat(id="gpt-4o"),
    success_criteria="A well-structured travel plan including flights, hotels, and activities when relevant.",
    instructions=[
        "Direct flight-related questions to FlightPlanner.",
        "Direct accommodation-related questions to HotelFinder.",
        "Direct activity-related questions to ActivitySuggester.",
        "If vague, use most up to date information you have or ask specific clarifications",
    ],
    show_tool_calls=True,
    show_members_responses=True,
    markdown=True,
    memory=memory,
)

# Run Agent

In [ ]:
print("Welcome to Travel Assistant! Type 'TERMINATE' to exit.")


travel_team.print_response("I want to travel to Rome from SFO in 6 months", stream=True)

travel_team.print_response("Can you find me the chepeat flights from JFK to London?", stream=True)

![Agno Agent Traces](https://storage.googleapis.com/arize-phoenix-assets/assets/images/phoenix-docs-images/agno-orchestrator-framework.png)

# Evaluate Agno Travel Agent

In [ ]:
from phoenix.client import AsyncClient

px_client = AsyncClient()
primary_df = await px_client.spans.get_spans_dataframe(
    project_identifier="agno-orchestrator-worker-agent"
)
primary_df

In [ ]:
import pandas as pd

trace_df = primary_df.groupby("context.trace_id").agg(
    {
        "attributes.input.value": "first",
        "attributes.output.value": lambda x: " ".join(x.dropna()),
    }
)

trace_df.head()

## Agent Tool Calling Eval

In [ ]:
TOOL_CALLING_ORDER = """
You are evaluating the correctness of the tool calling order in an LLM application's trace.

You will be given:
1. The user input that initiated the trace
2. The full trace output, including the sequence of tool calls made by the agent

##
User Input:
{attributes.input.value}

Trace Output:
{attributes.output.value}
##

Respond with exactly one word: `correct` or `incorrect`.
1. `correct` →
- The tool calls occur in the appropriate order to fulfill the user's request logically and effectively.
- A proper answer involves calls to reviews, summaries, and recommendations where relevant.
2. `incorrect` → The tool calls are out of order, missing, or do not follow a coherent sequence for the given input.
"""

In [ ]:
import nest_asyncio

from phoenix.evals import OpenAIModel, llm_classify
from phoenix.trace import suppress_tracing

nest_asyncio.apply()

model = OpenAIModel(
    api_key=os.environ["OPENAI_API_KEY"],
    model="gpt-4.1",
    temperature=0.0,
)

rails = ["correct", "incorrect"]

with suppress_tracing():
    tool_eval_results = llm_classify(
        data=trace_df,
        template=TOOL_CALLING_ORDER,
        model=model,
        rails=rails,
        provide_explanation=True,
        verbose=False,
    )

tool_eval_results

In [ ]:
root_spans = primary_df[primary_df["parent_id"].isna()][["context.trace_id", "context.span_id"]]

tool_eval_results = tool_eval_results[["label", "explanation"]]

# Merge tool correctness eval results with trace_df
tool_correctness_df = pd.merge(
    trace_df, tool_eval_results, left_index=True, right_index=True, how="left"
)

# Merge with root spans to get valid span IDs
tool_correctness_df = pd.merge(
    tool_correctness_df.reset_index(), root_spans, on="context.trace_id", how="left"
).set_index("context.span_id", drop=False)

### Log Results to Phoenix

In [ ]:
# Log to Phoenix
await px_client.annotations.log_span_annotations_dataframe(
    dataframe=tool_correctness_df, annotation_name="Tool Correctness", annotator_kind="LLM"
)

![Eval Results](https://storage.googleapis.com/arize-phoenix-assets/assets/images/agno-agents-orchestrator-eval.png)